In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization , GRU
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

In [2]:
from tensorflow.keras.models import Sequential, model_from_json
import numpy as np
json_file = open('CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("best_model1_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
import pickle

with open('scaler2.pickle', 'rb') as f:
    scaler2 = pickle.load(f)
    
with open('encoder2.pickle', 'rb') as f:
    encoder2 = pickle.load(f)

    
print("Done")  

Done


In [4]:
import librosa


In [5]:
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)

def rmse(data, frame_length=2048, hop_length=512):
    # Compute the spectrogram
    spec = np.abs(librosa.stft(data, n_fft=frame_length, hop_length=hop_length))

    # Calculate RMS from the spectrogram
    rmse = librosa.feature.rms(S=spec, frame_length=frame_length, hop_length=hop_length)

    return np.squeeze(rmse)

def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    # Compute the spectrogram
    spec = np.abs(librosa.stft(data, n_fft=frame_length, hop_length=hop_length))

    # Calculate MFCCs from the spectrogram
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(spec), sr=sr)

    return np.squeeze(mfcc.T) if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])
    
    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [6]:
def get_predict_feat(path):
    d, s_rate= librosa.load(path, duration=2.5, offset=0.6)
    res=extract_features(d)
    result=np.array(res)
    result=np.reshape(result,newshape=(1,2376))
    i_result = scaler2.transform(result)
    final_result=np.expand_dims(i_result, axis=2)
    
    return final_result

In [7]:
emotions1={1:'Neutral', 2:'Sad', 3:'Happy', 4:'Angry'}
def prediction(path1):
    res=get_predict_feat(path1)
    predictions=loaded_model.predict(res)
    y_pred = encoder2.inverse_transform(predictions)
    print(y_pred[0][0]) 

In [8]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_02/03-01-02-02-02-02-02.wav")

1/1 [==============================] - 10s 10s/step
neutral


In [9]:
import IPython.display as ipd
from IPython.display import Audio

data,sr = librosa.load("Z:/dev/machine/audio_speech_actors_01-24/Actor_07/03-01-03-01-01-01-07.wav")

ipd.Audio(data,rate=sr)

In [10]:
from pvrecorder import PvRecorder
import wave, struct

In [11]:
for index, devices in enumerate(PvRecorder.get_available_devices()):
    print(f"[{index}] {devices}")

[0] Microphone (Realtek(R) Audio)


In [476]:
import time
import wave
import struct
from pvrecorder import PvRecorder


# Create PvRecorder instance
recorder = PvRecorder(device_index=0, frame_length=2048)

# Define recording duration in seconds
recording_duration = 4

# Define storage for audio frames
audio = []

path = 'audio/audio_recording.wav'

try:
    recorder.start()
    start_time = time.time()

    while time.time() - start_time < recording_duration:
        frame = recorder.read()
        audio.extend(frame)

except KeyboardInterrupt:
    pass
finally:
    recorder.stop()
    with wave.open(path, "w") as f:
        f.setparams((1, 2, 16000, len(audio), "NONE", "NONE"))
        f.writeframes(struct.pack("h" * len(audio), *audio))
    recorder.delete()

data,sr = librosa.load("audio/audio_recording.wav")


ipd.Audio(data,rate=sr)



In [477]:
prediction("audio/audio_recording.wav")

1/1 [==============================] - 0s 17ms/step
sad


In [370]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav")

1/1 [==============================] - 0s 20ms/step
neutral


In [15]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-01-01.wav")

1/1 [==============================] - 0s 43ms/step
neutral


In [16]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-03-01-01-01-01.wav")

1/1 [==============================] - 0s 44ms/step
happy


In [26]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-04-01-01-01-01.wav")

1/1 [==============================] - 0s 44ms/step
sad


In [27]:
prediction("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-04-01-02-02-01.wav")

1/1 [==============================] - 0s 45ms/step
sad


In [28]:
data,sr = librosa.load("Z:/dev/machine/audio_speech_actors_01-24/Actor_01/03-01-04-01-02-02-01.wav")


ipd.Audio(data,rate=sr)